In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import math
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date


In [2]:
#read in the data
df= pd.read_csv("vaclog", sep='\t', index_col=False)
df.drop(columns=["I_emission","I_grid","T-platinum"], inplace=True)


In [3]:
#add elapsed time

timestamps=pd.to_datetime(df["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
df.insert(1,"Elapsed time",runtime)
df.head()


,Live comments,Elapsed time,Time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-CERNOX
0,NaN,0.0,14/12/2022 15:09:10,-0.042424,3.310000e-09,9.550000e-10,4.996000e-09,5.204000e-07,0.35245,0.0,161.391
1,NaN,7.0,14/12/2022 15:09:17,-0.041634,3.310000e-09,9.550000e-10,4.996000e-09,5.204000e-07,0.35245,0.0,161.054
2,NaN,15.0,14/12/2022 15:09:25,-0.041559,3.310000e-09,9.560000e-10,4.996000e-09,5.198000e-07,0.35245,0.0,161.054
3,NaN,22.0,14/12/2022 15:09:32,-0.045949,3.310000e-09,9.570000e-10,4.996000e-09,5.198000e-07,0.35245,0.0,161.279
4,NaN,29.0,14/12/2022 15:09:39,-0.042498,3.320000e-09,9.590000e-10,4.996000e-09,5.198000e-07,0.35246,0.0,161.166


In [4]:
#Temperature curve for CERNOX 
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [5]:
#Plotting the general pressure evolution for overview
fig, ax = plt.subplots()

l5 = ax.plot(df["Elapsed time"]/3600, polyfit(A,df["T-CERNOX"]),marker=".", color="cyan", markersize=5,label='Temperature')
ax.set_ylabel('Temperature (K)')

ax.set_xlabel('Elapsed time (h)')
ax2=ax.twinx()

l1 = ax2.plot((df["Elapsed time"]/3600),df['Barion_1']/0.46,marker=".", markersize=5,label='BA1')
l2 = ax2.plot((df["Elapsed time"]/3600),df['Barion_2']/0.46,marker=".", markersize=5,label='BA2')
l3 = ax2.plot((df["Elapsed time"]/3600),df['DUAL experiment'],marker=".", markersize=5,label='Dual gauge')
l4 = ax2.plot((df["Elapsed time"]/3600),df['injection 100mbar'],marker=".", markersize=5,label='100mbar F-R')
ax2.set_ylabel('Pressure (mbar)')
#TODO - annotations
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 10,
        }
plt.text(14.8, 7.36e-6, 'Pressure rise due to LHe refill \u2192 ', fontdict = font)
plt.text(21, 1e-5, '~Saturated vapor pressure', fontdict = font)
ax2.legend(handles = l1+l2+l3+l4+l5, loc="upper right")
plt.title("Pressure evolution + temperatures. Correction for H2 applied.")
ax2.set_yscale('log')


In [6]:
#slicing the dataset
ramp = df.iloc[list((df["Elapsed time"]/3600).between(1.19,40))]
print(ramp)

      Live comments  Elapsed time                 Time  injection 100mbar  \
597             NaN        4285.0  14/12/2022 16:20:35             96.270   
598             NaN        4292.0  14/12/2022 16:20:42             96.271   
599             NaN        4299.0  14/12/2022 16:20:49             96.270   
600             NaN        4307.0  14/12/2022 16:20:57             96.270   
601             NaN        4314.0  14/12/2022 16:21:04             96.274   
...             ...           ...                  ...                ...   
12814           NaN       91982.0  15/12/2022 16:42:12             89.048   
12815           NaN       91989.0  15/12/2022 16:42:19             89.048   
12816           NaN       91996.0  15/12/2022 16:42:26             89.048   
12817           NaN       92004.0  15/12/2022 16:42:34             89.048   
12818           NaN       92011.0  15/12/2022 16:42:41             89.048   

           Barion_2      Barion_1  DUAL experiment  DUAL insulation  \
597 

In [7]:
#plotting the sliced data
#Plotting the general pressure evolution for overview
fig, ax = plt.subplots()

ax.plot((ramp["Elapsed time"]/3600),ramp['Barion_1']/0.46,marker=".", markersize=5,label='BA1')
ax.plot((ramp["Elapsed time"]/3600),ramp['Barion_2']/0.46,marker=".", markersize=5,label='BA2')
ax.plot((ramp["Elapsed time"]/3600),ramp['DUAL experiment'],marker=".", markersize=5,label='DUAL gauge')
plt.legend(loc="upper right")

ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 12,
        }
plt.text(14.25, 6e-6, 'Pressure rise due to LHe refill \u2192 ', fontdict = font)
plt.text(13.02, 4.32e-6, 'Temperature fluctuation \u2192', fontdict = font)
plt.legend(loc="upper right")
plt.title("Pressure evolution during injection. Corrected for H2")


Text(0.5, 1.0, 'Pressure evolution during injection. Corrected for H2')

In [8]:
#TODO - calculating coverages for plotting pressure v coverage
#in the first assumption coverage is assumed to be equivalent to the number of molecules injected
#coverage at any given time point in [M/cm2] -> M/s * total injection time 
#First - Determine the inj start point, set elapsed time here to 0. 
inj = df.loc[df["Barion_2"]/0.46 >= 9e-8]
inj.drop(columns="Elapsed time", inplace=True)
print(inj)
timestamps2 = pd.to_datetime(inj["Time"],format="%d/%m/%Y %H:%M:%S")
runtime2 = (timestamps2-timestamps2[633]).dt.total_seconds()    
inj.insert(1,"Elapsed time",runtime2)



      Live comments                 Time  injection 100mbar      Barion_2  \
633             NaN  14/12/2022 16:24:53             96.302  1.550000e-07   
634             NaN  14/12/2022 16:25:01             96.298  1.680000e-07   
635             NaN  14/12/2022 16:25:08             96.298  2.340000e-07   
636             NaN  14/12/2022 16:25:15             96.294  1.040000e-07   
637             NaN  14/12/2022 16:25:22             96.295  6.830000e-08   
...             ...                  ...                ...           ...   
12814           NaN  15/12/2022 16:42:12             89.048  3.950000e-06   
12815           NaN  15/12/2022 16:42:19             89.048  3.940000e-06   
12816           NaN  15/12/2022 16:42:26             89.048  3.940000e-06   
12817           NaN  15/12/2022 16:42:34             89.048  3.930000e-06   
12818           NaN  15/12/2022 16:42:41             89.048  3.930000e-06   

           Barion_1  DUAL experiment  DUAL insulation  injection 1mbar  \
6

C:\Users\einot\AppData\Local\Temp\ipykernel_13716\718386385.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inj.drop(columns="Elapsed time", inplace=True)


## Subtracting baseline

In [9]:
#TODO - subtract the baseline from graph 2 (linear pressure scale) and plot again
#adding H2 equiv pressure readings
inj.insert(3, "corr BA2", inj["Barion_2"]/0.46)
inj.insert(4, "corr BA1", inj["Barion_1"]/0.46)


### Thermal transpiration
Applied for the gauge readings using the formula below:
    $$
    \frac{p_2}{p_1}=\sqrt \frac{T_2}{T_1}
    $$
Here $p_2$ and $T_2$ are the pressure and temperature in the cold part and $p_1$, $T_1$ are the pressure, temperature the gauge is exposed to.

In [10]:
#implementing the formula
T2 = 4.2
T1 = 293
p_coef = np.sqrt(T2/T1)
inj.insert(3, "H2 4K BA2", (inj["corr BA2"]*p_coef))
inj.insert(4, "H2 4K BA1", (inj["corr BA1"]*p_coef))


In [11]:
print(p_coef)
inj.head()

0.11972665112564158


,Live comments,Elapsed time,Time,H2 4K BA2,H2 4K BA1,corr BA2,corr BA1,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-CERNOX
633,NaN,0.0,14/12/2022 16:24:53,4.034268e-08,4.008240e-08,3.369565e-07,3.347826e-07,96.302,1.550000e-07,1.540000e-07,1.336000e-07,3.305000e-07,1.1,407.0,4281.128
634,NaN,8.0,14/12/2022 16:25:01,4.372626e-08,4.710983e-08,3.652174e-07,3.934783e-07,96.298,1.680000e-07,1.810000e-07,1.935000e-07,3.309000e-07,1.1,408.0,4281.240
635,NaN,15.0,14/12/2022 16:25:08,6.090443e-08,6.012360e-08,5.086957e-07,5.021739e-07,96.298,2.340000e-07,2.310000e-07,1.139000e-07,3.309000e-07,1.1,408.0,4281.240
636,NaN,22.0,14/12/2022 16:25:15,2.706863e-08,2.654808e-08,2.260870e-07,2.217391e-07,96.294,1.040000e-07,1.020000e-07,7.460000e-08,3.309000e-07,1.1,408.0,4281.240
637,NaN,29.0,14/12/2022 16:25:22,1.777680e-08,1.746447e-08,1.484783e-07,1.458696e-07,96.295,6.830000e-08,6.710000e-08,4.528000e-08,3.309000e-07,1.1,408.0,4281.464


In [12]:
#define baseline
base1 = inj[inj["corr BA1"].between(1e-7,2e-7)]
base2 = inj[inj["corr BA2"].between(1e-7,2e-7)]

#take the average of the baseline and apply thermal transpiration coef to these values
avg2 = np.mean(base1["corr BA2"])*p_coef
avg1 = np.mean(base2["corr BA1"])*p_coef
print(avg1)
print(avg2)


1.563379132438882e-08
1.5810932988391844e-08


In [13]:
#subtract the baseline from the graph to get the actual pressure evolution on the sample

#Plotting the h2 equiv base pressure subtracted data
fig, ax = plt.subplots()
#subtract the baseline from the graph to get the actual pressure evolution on the sample

ax.plot((inj["Elapsed time"]/3600),inj['H2 4K BA1'].sub(avg1),marker=".", markersize=5,label='BA1')
ax.plot((inj["Elapsed time"]/3600),inj['H2 4K BA2'].sub(avg2),marker=".", markersize=5,label='BA2')
plt.legend(loc="upper left")

ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')

plt.legend(loc="upper right")
plt.title("Pressure evolution on the sample, p increase from inj flux subtracted. \n Pressures H2 equivalent, thermal transp. corrected ")


Text(0.5, 1.0, 'Pressure evolution on the sample, p increase from inj flux subtracted. \n Pressures H2 equivalent, thermal transp. corrected ')

### Injection pressure linearity

In [14]:
#defining a function for the straight line
def func(x,a,b):
    return a*x + b
#curve fit for the data
y = inj["injection 100mbar"].values
x = (inj["Elapsed time"]/3600).values
params, cov = curve_fit(func,x,y)
#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}")

Fitted line: y = -0.30*x+96.47


In [15]:
#checking inj pressure linearity, graph
fig, ax = plt.subplots()
y_fit = func(x,a,b)
ax.plot((inj["Elapsed time"]/3600),inj["injection 100mbar"],marker=".", markersize=5,label='inj volume pressure')
ax.plot((inj["Elapsed time"]/3600),y_fit, "--",color="red", label="Fit") 
ax.legend(loc="upper right")

ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')
plt.title("Injection pressure evolution 100mbar conductance gauge")

Text(0.5, 1.0, 'Injection pressure evolution 100mbar conductance gauge')

### Formula for sum of the differential injection flux
$$
N=\frac{\Sigma dp\cdot V_{inj}}{k_B\cdot T}=\frac{\Sigma (p_{inj,t(x)}-p_{inj,t(x-1)})\cdot V_{inj}}{k_B \cdot T}   \space \left[{M}\right]
$$

In [16]:
#constants in SI units
T = 293
k_B = 1.38e-23
V_inj = 6.515e-5
S_sample= 276
#implement the formula above
dp = abs(np.diff(inj["injection 100mbar"]*100))
print(np.mean(dp))
#dt = np.diff(inj["Elapsed time"])
#dif = abs(dp)*V_inj/dt
dp = np.insert(dp,0,0)
N_x = (dp.cumsum())*V_inj/(k_B*T)
print(N_x)
N_cov = (N_x/S_sample)
print(N_cov)


0.0940838126540632
[0.00000000e+00 6.44507098e+15 6.44507098e+15 ... 1.84490157e+19
 1.84490157e+19 1.84490157e+19]
[0.00000000e+00 2.33517064e+13 2.33517064e+13 ... 6.68442597e+16
 6.68442597e+16 6.68442597e+16]


In [17]:
inj.insert(2, "Coverage",N_cov)


In [18]:
inj.head()

,Live comments,Elapsed time,Coverage,Time,H2 4K BA2,H2 4K BA1,corr BA2,corr BA1,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-CERNOX
633,NaN,0.0,0.000000e+00,14/12/2022 16:24:53,4.034268e-08,4.008240e-08,3.369565e-07,3.347826e-07,96.302,1.550000e-07,1.540000e-07,1.336000e-07,3.305000e-07,1.1,407.0,4281.128
634,NaN,8.0,2.335171e+13,14/12/2022 16:25:01,4.372626e-08,4.710983e-08,3.652174e-07,3.934783e-07,96.298,1.680000e-07,1.810000e-07,1.935000e-07,3.309000e-07,1.1,408.0,4281.240
635,NaN,15.0,2.335171e+13,14/12/2022 16:25:08,6.090443e-08,6.012360e-08,5.086957e-07,5.021739e-07,96.298,2.340000e-07,2.310000e-07,1.139000e-07,3.309000e-07,1.1,408.0,4281.240
636,NaN,22.0,4.670341e+13,14/12/2022 16:25:15,2.706863e-08,2.654808e-08,2.260870e-07,2.217391e-07,96.294,1.040000e-07,1.020000e-07,7.460000e-08,3.309000e-07,1.1,408.0,4281.240
637,NaN,29.0,5.254134e+13,14/12/2022 16:25:22,1.777680e-08,1.746447e-08,1.484783e-07,1.458696e-07,96.295,6.830000e-08,6.710000e-08,4.528000e-08,3.309000e-07,1.1,408.0,4281.464


In [19]:
fig, ax = plt.subplots()

ax.plot((inj["Elapsed time"]/3600),inj["Coverage"],marker=".", markersize=5,label='Coverage')
ax.legend(loc="upper right")

ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('Coverage (M/s*cm2)')
ax.set_yscale('linear')
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 12,
        }

plt.text(15.30, 8e+15, 'Slope change coherent with inj. pressure graph \u2192 ', fontdict = font)
plt.title("Coverage over time")

Text(0.5, 1.0, 'Coverage over time')

In [20]:
## dropping coverage values close to 0
inj_new = inj.loc[inj['Coverage'] > 1e+13]
inj_new.head()
#inj_new["H2 4K BA2"].sub(avg2).describe()

,Live comments,Elapsed time,Coverage,Time,H2 4K BA2,H2 4K BA1,corr BA2,corr BA1,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-CERNOX
634,NaN,8.0,2.335171e+13,14/12/2022 16:25:01,4.372626e-08,4.710983e-08,3.652174e-07,3.934783e-07,96.298,1.680000e-07,1.810000e-07,1.935000e-07,3.309000e-07,1.1,408.0,4281.240
635,NaN,15.0,2.335171e+13,14/12/2022 16:25:08,6.090443e-08,6.012360e-08,5.086957e-07,5.021739e-07,96.298,2.340000e-07,2.310000e-07,1.139000e-07,3.309000e-07,1.1,408.0,4281.240
636,NaN,22.0,4.670341e+13,14/12/2022 16:25:15,2.706863e-08,2.654808e-08,2.260870e-07,2.217391e-07,96.294,1.040000e-07,1.020000e-07,7.460000e-08,3.309000e-07,1.1,408.0,4281.240
637,NaN,29.0,5.254134e+13,14/12/2022 16:25:22,1.777680e-08,1.746447e-08,1.484783e-07,1.458696e-07,96.295,6.830000e-08,6.710000e-08,4.528000e-08,3.309000e-07,1.1,408.0,4281.464
638,NaN,36.0,5.837927e+13,14/12/2022 16:25:29,1.082745e-08,1.064526e-08,9.043478e-08,8.891304e-08,96.294,4.160000e-08,4.090000e-08,2.502000e-08,3.309000e-07,1.1,408.0,4281.464


In [22]:
#plotting pressure vs coverage
fig, ax = plt.subplots()

ax.plot((inj_new["Coverage"]),inj_new["H2 4K BA2"].sub(avg2),marker=".", markersize=5,label='isotherm')
ax.legend(loc="upper left")
xrange = np.arange(min(inj_new["Coverage"]),max(inj_new["Coverage"]),1e+16)
print(xrange)
ax.set_xticks(xrange)
plt.xlim(1e16,7e16)
#ax.set_xlim(left = 3e+15,right = 9.20e15)
#ax.set_xlim(right= 9.23e+15)
#ax.set_xlim(2.9e+12,9.23e+15)
#ax.set_ylim(1e-10,1.5e-6)
ax.yaxis.set_major_formatter(ScalarFormatter())
ax.minorticks_off()

ax.set_xlabel('Coverage (M/cm2)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('log')
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 10,
        }
#plt.text(5e+15, 7.1e-7, 'Pressure rise due to LHe refill \u2192 ', fontdict = font)
#plt.text(8.15e+15, 8.7e-7, '~Saturated vapor pressure', fontdict = font)
plt.title("H2 adsorption isotherm")

[2.33517064e+13 1.00233517e+16 2.00233517e+16 3.00233517e+16
 4.00233517e+16 5.00233517e+16 6.00233517e+16]


Text(0.5, 1.0, 'H2 adsorption isotherm')